In [45]:
# Finding Lane Lines for Videos
# import packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math 
%matplotlib inline
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# mask for seperating yellow and white colors
def convert_hls(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)

def select_white_yellow(image):
    converted = convert_hls(image)
    # white color mask
    lower = np.uint8([  0, 190,   0])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted, lower, upper)
    # yellow color mask
    lower = np.uint8([ 10,   0, 100])
    upper = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(converted, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    return cv2.bitwise_and(image, image, mask = mask)

# calculating average slope and intercept of a group of lines (seperate left and right)
def findAvgLaneLines(lines):
    left_lines    = [] # (slope, intercept)
    left_weight  = [] # (length)
    right_lines   = [] # (slope, intercept)
    right_weight = [] # (length)
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1 == x2: #ignore vertical lines, slope not defined
                continue
            m = (y2-y1)/(x2-x1)
            b = y1-m*x1
            if (abs(m) < 0.5):
                continue
            length = math.sqrt(math.pow((y2-y1),2)+math.pow((x2-x1),2))
            if m < 0: # left lines since y is reversed in image
                left_lines.append((m,b))
                left_weight.append((length))
            else:
                right_lines.append((m,b))
                right_weight.append((length))
    # add more weight to longer lines    
    left_lane  = np.dot(left_weight,  left_lines) /np.sum(left_weight)  if len(left_weight) >0 else None
    right_lane = np.dot(right_weight, right_lines)/np.sum(right_weight) if len(right_weight)>0 else None

    return left_lane, right_lane # (slope, intercept), (slope, intercept)

# taking in lines and making 2 points
def points_from_line(y1,y2,line):
    if line is None:
        return None
    
    m, b = line
    
    x1 = int((y1 - b)/m)
    x2 = int((y2 - b)/m)
    y1 = int(y1)
    y2 = int(y2)

    return ((x1, y1), (x2, y2))

# process the image and overlay lane lines
def process_image(image):
    
    white_yellow = select_white_yellow(image)
    
    # change image to greyscale
    gray = cv2.cvtColor(white_yellow,cv2.COLOR_RGB2GRAY)

    # define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # define parameters for Canny and apply
    low_threshold = 60
    high_threshold = 200
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # create masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # define four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(210,imshape[0]),(600, 450), (780, 450), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # define the Hough transform parameters
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 8  # minimum number of pixels making up a line
    max_line_gap = 12   # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # run Hough on edge detected image
    # output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
            
    # find average slope and y-intercept for lines
    avg_lane_lines = findAvgLaneLines(lines)

    # define the image for average lines and draw the average lines 
    overlay = image.copy()
    ybottom = imshape[0]
    ytop = 440
    points_left = points_from_line(ybottom, ytop, avg_lane_lines[0]) #((x1, y1), (x2, y2))
    points_right = points_from_line(ybottom, ytop, avg_lane_lines[1]) #((x1, y1), (x2, y2))
    if points_left is not None:
        cv2.line(overlay,points_left[0],points_left[1],(255,0,0),10)
        cv2.line(overlay,points_right[0],points_right[1],(255,0,0),10)

    # overlay the average lines over the original image with a transparent red color
    output = image.copy()
    processed = cv2.addWeighted(overlay, 0.4, output, 0.6, 0)
    return processed

challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))


[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:16<00:00, 15.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 9.4 s, sys: 2.97 s, total: 12.4 s
Wall time: 18.3 s
